In [22]:
import os
from dotenv import load_dotenv
import assemblyai as aai
from openai import OpenAI

In [ ]:
load_dotenv()
aai.settings.api_key = os.getenv("aai_key")

In [35]:
target       = "audio_session_4"
audio_file   = f"../recordings/{target}.wav"
speech_model = aai.SpeechModel.best
# speech_model = aai.SpeechModel.nano
n_speakers   = 3
language     = "es"

In [15]:
config = aai.TranscriptionConfig(
    speech_model      = speech_model,
    speaker_labels    = True,
    speakers_expected = n_speakers,
    language_code     = language
)

transcriber = aai.Transcriber()
transcript = transcriber.transcribe(
  audio_file,
  config = config
)

if transcript.status == aai.TranscriptStatus.completed:
    print(f"Transcription was completed successfully!")
    exit(1)
if transcript.status == aai.TranscriptStatus.error:
    print(f"Transcription failed: {transcript.error}")
    exit(1)

Transcription was completed successfully!


In [24]:
transcript_best = [f"Speaker {utterance.speaker}: {utterance.text}" for utterance in transcript.utterances]
full_transcript = "\n".join(transcript_best)

In [25]:
system_prompt = """
Eres un experto consultor en educación, especializado en tutorías y métodos de enseñanza efectivos. Recibirás 
transcripciones de sesiones de tutoría grabadas entre un profesor y un estudiante de escuela primaria. Tu tarea es analizar 
detalladamente la manera en que el profesor explica los conceptos abordados durante cada sesión. Específicamente, debes:

1. Evaluar las técnicas de explicación: Analizar la claridad, la estructura y la precisión de las explicaciones. 
Identificar los puntos fuertes y las áreas de mejora en el uso del lenguaje, ejemplos, analogías y estrategias de enseñanza.
Determinar si el profesor adapta su explicación de forma adecuada al nivel de comprensión de un estudiante de escuela primaria.

2. Evaluar el compromiso y la eficacia pedagógica: Valorar cómo el profesor involucra al estudiante y verifica su comprensión 
a lo largo de la sesión. Considerar si se emplean métodos interactivos o reflexivos que refuercen la asimilación de los conceptos.

3. Proporcionar retroalimentación constructiva: Ofrecer sugerencias detalladas y accionables para mejorar los métodos de 
enseñanza del profesor. Resaltar las mejores prácticas, tales como simplificar conceptos complejos, usar ejemplos relevantes y claros, 
resumir los puntos clave y fomentar la participación activa del estudiante. Asegurarte de que la retroalimentación sea objetiva, alentadora 
y orientada al crecimiento profesional del profesor.

4. Limitaciones y contexto: Basar tu análisis únicamente en el contenido proporcionado en la transcripción. Indicar, si es necesario, 
qué información adicional podría ayudar a realizar una evaluación más completa.

Tu respuesta debe estar organizada, ser clara y exhaustiva, reflejando las mejores prácticas en pedagogía y en la retroalimentación 
constructiva.
"""

In [29]:
context_prompt = f"""
La siguiente es una transcripción de una sesión de tutoría entre un profesor y un estudiante de escuela primaria:

[INICIO DE LA TRANSCRIPCIÓN]

{full_transcript}

[FIN DE LA TRANSCRIPCIÓN]

La transcripción puede contener errores y omisiones. Por lo tanto, no te enfoques en errores de gramática o typos. Así mismo,
la transcripción divide las intervenciones por interlocutor pero no especifica específicamente quién es el profesor y quién es el estudiante.
Por lo tanto, deberás inferir quién es quién basado en el contenido de las intervenciones.

Basado en la transcripción proveída, por favor evalúa las técnicas de explicación, el compromiso y la eficacia pedagógica del profesor
y prepara un set estructurado de recomendaciones para mejorar sus métodos de enseñanza. Asegúrate de ser específico y detallado en tu análisis.
Así mismo, asegurate de que tu retroalimentación sea objetiva, alentadora y orientada al crecimiento profesional del profesor. Resalta tanto
aspectos positivos como negativos basados en tu evaluación y tu conocimiento de mejores prácticas pedagógicas.

Para obtener una retroalimentación valiosa, toma en cuenta los siguientes aspectos claves:

1. Estructura y organización de la tutoría:
- ¿Hay una introducción clara del tema y objetivos de aprendizaje?
- ¿La secuencia de contenidos y ejercicios sigue un orden lógico?
- ¿Se hace un cierre efectivo que consolide el aprendizaje?

2. Interacción profesor-estudiante:
- ¿Qué tan frecuente es la participación del estudiante?
- ¿Cómo responde el tutor a las preguntas y dudas?
- ¿Se verifica la comprensión del estudiante regularmente?

3. Técnicas pedagógicas:
- ¿Qué estrategias usa para explicar conceptos complejos?
- ¿Se emplean ejemplos relevantes y contextualizados?

4. Manejo del tiempo:
- ¿Se dedica suficiente tiempo a cada tema?
- ¿Se ajusta el ritmo según la comprensión del estudiante?
- ¿Se aprovecha efectivamente el tiempo disponible?

5. Aspectos específicos a evaluar:
- Claridad de las explicaciones
- Capacidad para identificar y abordar confusiones
- Habilidad para mantener el interés y motivación
- Uso de preguntas guía y técnicas de andamiaje
- Adaptabilidad a las necesidades del estudiante

Por favor, proporciona una retroalimentación detallada y constructiva que ayude al profesor a mejorar su desempeño pedagógico en su 
próxima sesión. gracias.
"""

In [30]:
history = [
    {"role": "system", "content": system_prompt},
    {"role": "user",   "content": context_prompt}
]

In [56]:
system_prompt

'\nEres un experto consultor en educación, especializado en tutorías y métodos de enseñanza efectivos. Recibirás \ntranscripciones de sesiones de tutoría grabadas entre un profesor y un estudiante de escuela primaria. Tu tarea es analizar \ndetalladamente la manera en que el profesor explica los conceptos abordados durante cada sesión. Específicamente, debes:\n\n1. Evaluar las técnicas de explicación: Analizar la claridad, la estructura y la precisión de las explicaciones. \nIdentificar los puntos fuertes y las áreas de mejora en el uso del lenguaje, ejemplos, analogías y estrategias de enseñanza.\nDeterminar si el profesor adapta su explicación de forma adecuada al nivel de comprensión de un estudiante de escuela primaria.\n\n2. Evaluar el compromiso y la eficacia pedagógica: Valorar cómo el profesor involucra al estudiante y verifica su comprensión \na lo largo de la sesión. Considerar si se emplean métodos interactivos o reflexivos que refuercen la asimilación de los conceptos.\n\n3

In [32]:
client = OpenAI()

chat_completion = client.chat.completions.create(
    messages = history,
    model    = "gpt-4o-2024-08-06"
)

In [40]:
print(chat_completion.choices[0].message.content)

**Análisis de la Sesión de Tutoría**

**1. Evaluación de las Técnicas de Explicación:**

- **Claridad y estructura de las explicaciones**: El profesor en la transcripción utiliza un lenguaje sencillo y directo, adecuado para estudiantes de escuela primaria. La estructura básica de cada ejercicio es clara: se presentan los números, se invitan a los estudiantes a identificar las centenas, decenas y unidades, y se confirma la respuesta. Este enfoque repetitivo ayuda a reforzar el concepto de valor posicional.

- **Uso de ejemplos y analogías**: Se emplean ejemplos contextuales acertados, como la comparación de números con cajas y grupos (cajas de 100, cajitas de 10, y sueltos). Esto es efectivo para que el estudiante relacione conceptos matemáticos abstractos con situaciones cotidianas.

- **Adaptación al nivel del estudiante**: La adaptación parece adecuada, utilizando un lenguaje simple y dinámico para mantener la atención y comprensión del estudiante. Sin embargo, sería beneficioso div

In [41]:
with open(f"../transcripts/{target}_transcript.txt", "w") as f:
    f.write(full_transcript)

with open(f"../transcripts/{target}_feedback.txt", "w") as f:
    f.write(chat_completion.choices[0].message.content)